# 유한 오토마타 Finite Automata

오토마타를 "상태 기계"(State Machine)라고도 한다. 따라서 유한 오토마타(Finte Automata; 줄여서 FA)는 "유한 상태 기계"(Finate State Macheime; 줄여서 FSM)라고도 한다. 여기서는 오토마타라는 이름으로 부르겠다. 유한 오토마타는 줄여서 FA라고 쓰겠다.

FA는 라벨된 방향 그래프(labeled directed graph)로 생각하면 된다. 그래프의 노드가 상태에 해당하고 화살표에 알파벳 라벨이 붙어 있으며, 하나의 시작 상태(initial state)가 있고 어러개(0개 이상)의 종료 상태(final state)가 있다. FA는 시작상태에서 화살표를 따라서 여러번 진행하여 종료상태에 도달하기까지 지나온 화살표의 라벨을 순서대로 나열한 문자열(글줄)을 받아들이는(accepts) 기계이다.
FA는 시작상태에서 종료상태로 갈 수 있는 가능한 모든 경로에 해당하는 문자열의 집합, 즉 어떤 언어를 규정한다고 볼 수 있다.

이론적으로 흥미로운 사실은 앞서 배운 정규식(RE)로 정의할 수 있는 언어와
여기서 다루는 유한 오토마타(FA)로 규정할 수 있는 언어의 범위가 정확히 일치한다는 점이다. 즉,
 * 어떤 RE로 정의되는 언어를 규졍하는 FA를 구성할 수 있으며 
 * 어떤 FA로 규정되는 언어를 정의하는 RE를 작성할 수 있다.

실용적인 관점에서는 전자가 더 흥미롭다.
왜내하면 RE는 수학적 표기에 가깝지만 FA는 바로 알고리듬 즉 프로그램으로 작성하기에 알맞은 구조이기 때문이다.
RE는 사람이 읽기 편한 수학적 표기법이며 어떤 성질의 언어인지 그 언어의 집합을 정의하기에 알맞은 구조이다.
하지만 주어진 문자열이 언어에 속하는지 아닌지 검사하는 알고리듬을 어떻게 작성할지에 대해서는
RE의 문법구조가 직접적인 힌트가 되지는 못한다.
반면 FA는 사람이 읽기에는 좀 불친절하지만 시작상태로부터 상태전이 화살표를 따라가면 종료상태에 도달하는지 검사하는 알고리듬으로 옮기기 쉬운 명세이다.
따라서 RE로 정의되는 언어를 규정하는 FA를 자동으로 생성하여 주어진 문자열이
그 FA가 받아들이는 문자열인지 검사함으로써 어떤 문자열이 RE가 표현하는 언어에 속하는지 검사하는 프로그램을 작성할 수 있다.

## DFA, NFA, NFA-$\epsilon$
보통 형식언어 및 오토마타를 주로 다루는 교재에서는 다음과 같이 네 단계를 거쳐 변환하여 상태가 최소화된 DFA를 얻어낸다.

RE $\xrightarrow{\qquad\qquad}$
NFA-$\epsilon$ $\xrightarrow{\quad\epsilon~전이~제거\quad}$
NFA $\xrightarrow{\qquad\qquad}$
DFA $\xrightarrow{\quad상태~최소화\quad}$
DFA

이렇게 하면 상태가 최소화되어 효율적인 정규언어 검사기를 만들 수 있으며
실제로 구문분석기(lexer)를 생성하는 프로그램들이 이러한 방식으로 만들어져 있다.
여기서는 RE와 NFA에 대해서만 다룰 것이므로 DFA나 NFA-$\epsilon$을 비롯해서 위 그림에 대한
자세한 내용은 형식언어/오토마타/계산이론 관련 교재를 참고하여 시간이 날 때 틈틈이 스스로 공부해 두는 것이 좋다.
참고로 DFA, NFA, NFA-$\epsilon$ 세 가지 종류의 FA가 규정할 수 있는 언어의 범위가 정확히 일치한다.
그래서 표면적으로 보기에는 더 많은 기능을 가진 NFA-$\epsilon$로부터 $\epsilon$-전이 기능을 제외한 NFA로
그리고 비결정성을 제외한 DFA로의 변환이 일반적으로 가능하다.

보통 컴퓨터 관련 전공 학과에서 프로그래밍 언어나 컴파일러 등의 선수과목으로
형식언어/오토마타/계산이론을 다루는 과목이 있는 것이 이론을 탄탄히 익히기에 바람직한데,
아쉽게도 우리 학과에는 따로 없다. 그래서 이 과목 초반부에 최단시간 안에 함수형 프로그래밍과
정규언어 관련 내용을 같이 익혀야 하는 상황이다.
그러니까 단계를 줄여서 RE에서 NFA까지 그냥 한꺼번에 가고
그 다음 단계들 없이, 즉 NFA를 DFA로 변환하지 않고 그냥 NFA 명세를 이용해 바로 실행하는 Haskell 프로그램을 작성해 보도록 하자.

## NFA의 하스켈 타입 정의와 상태 전이 함수

In [1]:
-- 상태(State)와 라벨(Label) 타입을 고정한 NFA 정의
type State = Int
type Label = Char
type Delta = [(State, Label, State)] -- 관계 집합(리스트)로 상태 전이

-- NFA의 초기상태는 항상 0으로 고정하고 표기하지 않는다. 편의상 state 의 max 값을 저장한다
type NFA = (State, Delta, [State]) -- (max state, final states, transition relation)

-- 관계 집합으로부터 상태 전이 함수 정의
delta :: NFA -> (State -> Label -> [State])
delta (_,ds,_) = \ st lb -> [q | (p,l,q) <- ds, p == st, l == lb]

-- Emtpy 정규식에 해당하는 NFA 생성
emptyNFA :: NFA
emptyNFA = (0,[],[])
-- Epsilon 정규식에 해당하는 NFA 생성
epsilonNFA :: NFA
epsilonNFA = (0,[],[0])
-- Alphabet c 정규식에 해당하는 NFA 생성
alphaNFA :: Label -> NFA
alphaNFA c = (1,[(0,c,1)],[1])

In [2]:
emptyNFA
epsilonNFA
alphaNFA 'a'

(0,[],[])

(0,[],[0])

(1,[(0,'a',1)],[1])

In [3]:
import Data.List (union)

mapNFAstate :: (State -> State) -> NFA -> NFA
mapNFAstate f (m,ds,qs) = (f m, [(f p, c, f q) | (p,c,q)<-ds], map f qs)

unionNFA :: NFA -> NFA -> NFA
unionNFA nfa1@(m1,ds1,qs1) nfa2 = (m2, union ds1 ds2, union qs1 qs2)
  where
  (m2,ds2,qs2) = mapNFAstate f nfa2
  f 0 = 0
  f x = x + m1

concatNFA :: NFA -> NFA -> NFA
concatNFA = undefined

kleeneNFA :: NFA -> NFA
kleeneNFA = undefined

In [4]:
alphaNFA 'a'
alphaNFA 'b'
mapNFAstate (\x -> case x of {0 -> 0; x -> x + 1}) (alphaNFA 'b')
unionNFA (alphaNFA 'a') (alphaNFA 'b')

(1,[(0,'a',1)],[1])

(1,[(0,'b',1)],[1])

(2,[(0,'b',2)],[2])

(2,[(0,'a',1),(0,'b',2)],[1,2])

In [5]:
accepts :: NFA -> String -> Bool
accepts nfa@(_,_,finals) str = any (`elem` finals) (steps 0 str) 
  where
  step :: State -> Label -> [State]
  step = delta nfa
  steps :: State -> [Label] -> [State]
  steps p []     = return p
  steps p (c:cs) = do q <- step p c
                      steps q cs

In [6]:
accepts (unionNFA (alphaNFA 'a') (alphaNFA 'b')) ""
accepts (unionNFA (alphaNFA 'a') (alphaNFA 'b')) "a"
accepts (unionNFA (alphaNFA 'a') (alphaNFA 'b')) "b"
accepts (unionNFA (alphaNFA 'a') (alphaNFA 'b')) "c"
accepts (unionNFA (alphaNFA 'a') (alphaNFA 'b')) "aa"
accepts (unionNFA (alphaNFA 'a') (alphaNFA 'b')) "ab"
accepts (unionNFA (alphaNFA 'a') (alphaNFA 'b')) "ba"
accepts (unionNFA (alphaNFA 'a') (alphaNFA 'b')) "bb"

False

True

True

False

False

False

False

False

## 주어진 문자열이 정규식을 만족하는지 검사하는 프로그램

In [8]:
data RE -- 정규식 데이타 타입
  = Empty
  | Epsilon
  | Alphabet Char
  | Concat RE RE
  | Union RE RE
  | Kleene RE
  deriving Show

re2nfa :: RE -> NFA
re2nfa = undefined

accepts' :: RE -> String -> Bool
accepts' re str = re2nfa re `accepts` str